# Structure from motion example using a video

In [1]:
# Packages
import numpy as np
import cv2 as cv

## Load video and extract grey-scale images

In [2]:
video_file = r"C:\Fotos y videos\test SfM\video3.mp4"
video = cv.VideoCapture(video_file)

delta_t = 100 # ms

In [3]:
ret = True
current_time_ms = 0

images_list = []
while ret:
    video.set(cv.CAP_PROP_POS_MSEC, current_time_ms)
    ret, frame = video.read()
    key = cv.waitKey(0)
    if key == 27 or ret is False:
        break
    cv.imshow('press esc to stop', frame)
    images_list.append(cv.cvtColor(frame, cv.COLOR_BGR2GRAY))
    current_time_ms += delta_t
    if len(images_list) == 2:
        break

cv.destroyAllWindows()

### Display loaded images

In [4]:
for im in images_list:
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

## Extract image descriptors

In [5]:
sift = cv.SIFT_create()
kps_des_list = []
for im in images_list:
    kps, descriptors = sift.detectAndCompute(im, None) # No mask
    kps_des_list.append({'kps': kps,
                         'des': descriptors})
    

### Display found descriptors

In [6]:
for im, des_kps in zip(images_list, kps_des_list):
    im = cv.drawKeypoints(im, des_kps['kps'], im)
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

## Match found descriptos using k-nearest neighbours

In [7]:
# ratio of the nearest to the second nearest neighbour
ratio = 0.95

n_images = len(images_list)
matcher = cv.BFMatcher()

matches_list = []

for i in range(n_images - 1):
    good_matches = []
    matches = matcher.knnMatch(kps_des_list[i]['des'], kps_des_list[i+1]['des'], k=2)
    for m, n in matches:
        if m.distance < ratio*n.distance:
            good_matches.append(m)
    matches_list.append(good_matches)

### display good matches

In [8]:
for i, matches in enumerate(matches_list):
    im = cv.drawMatches(images_list[i], kps_des_list[i]['kps'], images_list[i+1], kps_des_list[i+1]['kps'], matches, im, 
                        flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

## Essential matrices estimation using RANSAC

In [9]:
# Camera matrix
camera_matrix = np.eye(3) # asume the center is in the upright corner and focal distance is 1 m
# f = images_list[0].shape[1]
# camera_matrix = np.array([[  f,   0,  images_list[0].shape[1]//2],
#                           [  0,   f,  images_list[0].shape[0]//2],
#                           [  0,   0,  1],])

ransac_threshold = 0.01

reproj_E_masks_list = []
masked_matches_list = []

for i in range(n_images - 1):
    src_points = np.array([kps_des_list[i]['kps'][m.queryIdx].pt for m in matches_list[i]], dtype='float32')
    dst_points = np.array([kps_des_list[i+1]['kps'][m.trainIdx].pt for m in matches_list[i]], dtype='float32') 
    E, mask = cv.findEssentialMat(src_points, dst_points, camera_matrix, method=cv.RANSAC, threshold=ransac_threshold, maxIters=5000)
    reproj_E_masks_list.append({"E": E, 
                                "mask": mask})
    # mask matches from reprojection error
    masked_matches = []
    for i_match in range(len(matches_list[i])):
        if mask[i_match] == 1:
            masked_matches.append(matches_list[i][i_match])
    masked_matches_list.append(masked_matches)


### Display matches after removing outliers

In [10]:
for i, matches in enumerate(matches_list):            
    im = cv.drawMatches(images_list[i], kps_des_list[i]['kps'], images_list[i+1], kps_des_list[i+1]['kps'], 
                        masked_matches_list[i], 
                        im, 
                        flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    cv.imshow('Press Esc to stop', im)
    key = cv.waitKey(0)
    if key == 27:
        break
cv.destroyAllWindows()

print([len(item) for item in masked_matches_list])

[20]


## Camera relative poses estimation

In [11]:

R_list, t_list = [], []
for i in range(n_images - 1):
    src_points = np.array([kps_des_list[i]['kps'][m.queryIdx].pt for m in matches_list[i]], dtype='float32')
    dst_points = np.array([kps_des_list[i+1]['kps'][m.trainIdx].pt for m in matches_list[i]], dtype='float32')
    retval, R, t, mask = cv.recoverPose(
        E=reproj_E_masks_list[i]['E'],
        points1=src_points, points2=dst_points, 
        mask=reproj_E_masks_list[i]['mask'], 
        cameraMatrix=camera_matrix)
    R_list.append(R)
    t_list.append(t)


# Triangulate points

In [13]:
points_3d_list = []
for i in range(n_images - 1):
    src_points = np.array([kps_des_list[i]['kps'][m.queryIdx].pt for m in masked_matches_list[i]], dtype='float')
    dst_points = np.array([kps_des_list[i+1]['kps'][m.trainIdx].pt for m in masked_matches_list[i]], dtype='float')
    
    projection_matrix1 = np.hstack((np.eye(3), np.zeros((3, 1))))
    projection_matrix2 = np.hstack((R_list[i], t_list[i]))
    
    points_4d = cv.triangulatePoints(projection_matrix1, projection_matrix2, src_points.T, dst_points.T)

    points_4d_homogeneous = points_4d.T / points_4d.T[:, 3].reshape((-1, 1))

    points_3d = cv.convertPointsFromHomogeneous(points_4d_homogeneous).reshape((-1,3))
    points_3d_list.append(points_3d)

points_3d_arr = np.vstack(points_3d_list)


### Remove outliers and plot!

In [14]:
import plotly.express as px
import pandas as pd

n_std = 1

center = np.mean(points_3d_arr, axis=0)
radius_display = np.std(points_3d_arr)*n_std

points_3d_arr_outliers_removed = np.array(
    [point for point in points_3d_arr if (np.sqrt(np.sum((point - center)**2)) < radius_display)]).reshape((-1, 3))

print(f"Showing only {points_3d_arr_outliers_removed.shape} of {points_3d_arr.shape} points")

df = pd.DataFrame(columns=['x', 'y', 'z'], data=points_3d_arr_outliers_removed)
fig = px.scatter_3d(df, x='x', y='y', z='z')
fig.show()

Showing only (17, 3) of (20, 3) points
